In [1]:
import tensorflow as tf
import numpy as np
import matplotlib
import os
from sklearn.preprocessing import MinMaxScaler

# 그래프 외부에 출력
%matplotlib tk

tf.set_random_seed(777)  # 어느 컴퓨터에서 이 코드를 실행해도 학습 방향이 같도록, 다시 수행해도 같도록

if "DISPLAY" not in os.environ:
    # remove Travis CI Error
    matplotlib.use('Agg')

import matplotlib.pyplot as plt

In [2]:
# train Parameters
seq_length = 5
data_dim = 2
hidden_dim = 10
output_dim = 1
learning_rate = 0.01
iterations =30

scaler = MinMaxScaler(feature_range=(0,1)) # 데이터 일반화
scaler2 = MinMaxScaler(feature_range=(0, 1)) # 데이터 일반화

xy = np.loadtxt('./teacher_data/train_data_v2_index.csv', delimiter=',')

x = scaler.fit_transform(xy[:, 0:-1])
y = scaler2.fit_transform(xy[:, [-1]]) 

# build a dataset
dataX = []
dataY = []

# for i in range(0, len(x) - seq_length): # 한 행씩 dataX, Y에 추가
#     _x = x[i:i + seq_length]
#     _y = y[i + seq_length]  # Next close price
#     dataX.append(_x)
#     dataY.append(_y)

for i in range(0, seq_length):
    tmp12 = []
    tmp122 = []
    for j in range(0, 30):
        tmp1 = x[j+((i*30)%150), 0:-1]
        tmp2 = y[j+((i*30)%150), [-1]]
        tmp12.append(tmp1)
        tmp122.append(tmp2)
    dataX.append(tmp12)
    dataY.append(tmp122)
    
xy2 = np.loadtxt('./teacher_data/test_data_v2_index.csv', delimiter=',') # 예측할 날짜

x2 = scaler.fit_transform(xy2) # 데이터 없을때 (데이터란 ptmt 값)
# x2 = scaler.fit_transform(xy2[:, 0:-1]) # 데이터 있을때
# y2 = scaler2.fit_transform(xy2[:, [-1]]) #데이터 있을때

dataX2 = []

# dataY2 = [] #데이터 있을때
for i in range(0, len(x2) - seq_length):
    _x2 = x2[i:i+seq_length]
#     _y2 = y2[i+seq_length] #데이터 있을때
    dataX2.append(_x2)
#     dataY2.append(_y2) #데이터 있을때

# train/test split

train_size = len(dataX)
test_size = len(dataX2)

# print(len(dataX))
# print(len(dataX2))

trainX, testX = np.array(dataX[0:train_size]),np.array(dataX2[0:test_size])
trainY = np.array(dataY[0:train_size])

print(np.shape(trainX))
print(np.shape(trainY))
print(np.shape(testX))

(5, 30, 1)
(5, 30, 1)
(5, 5, 2)


In [3]:
# input place holders
X = tf.placeholder(tf.float32, [None, seq_length, data_dim])
Y = tf.placeholder(tf.float32, [None, 1])

# build a LSTM network
cell = tf.contrib.rnn.BasicLSTMCell(
    num_units=hidden_dim, state_is_tuple=True, activation=tf.tanh)
outputs, _states = tf.nn.dynamic_rnn(cell, X, dtype=tf.float32)
Y_pred = tf.contrib.layers.fully_connected(
    outputs[:, -1], output_dim, activation_fn=None)  # We use the last cell's output

# cost/loss
loss = tf.reduce_sum(tf.square(Y_pred - Y))  # sum of the squares
# optimizer
optimizer = tf.train.AdamOptimizer(learning_rate)
train = optimizer.minimize(loss)

Instructions for updating:
Use the retry module or similar alternatives.


In [4]:
with tf.Session() as sess:
    init = tf.global_variables_initializer()
    sess.run(init)

    # Training step
    for i in range(iterations):
        _, step_loss = sess.run([train, loss], feed_dict={
                                X: trainX, Y: trainY})
        print("[step: {}] loss: {}".format(i, step_loss))
    # Test step
    test_predict = sess.run(Y_pred, feed_dict={X: testX})
    
    test_predict = scaler2.inverse_transform(test_predict)
#     dataY2 = scaler2.inverse_transform(dataY2) # 데이터 있을때

[step: 0] loss: 2022.7044677734375
[step: 1] loss: 1363.018798828125
[step: 2] loss: 895.7305908203125
[step: 3] loss: 606.277587890625
[step: 4] loss: 482.8088073730469
[step: 5] loss: 483.0735778808594
[step: 6] loss: 536.4558715820312
[step: 7] loss: 573.4605712890625
[step: 8] loss: 575.3316040039062
[step: 9] loss: 552.8365478515625
[step: 10] loss: 521.30810546875
[step: 11] loss: 491.998046875
[step: 12] loss: 470.81927490234375
[step: 13] loss: 459.4259338378906
[step: 14] loss: 456.6819152832031
[step: 15] loss: 459.95147705078125
[step: 16] loss: 466.11016845703125
[step: 17] loss: 472.32464599609375
[step: 18] loss: 476.5727233886719
[step: 19] loss: 477.86016845703125
[step: 20] loss: 476.1408996582031
[step: 21] loss: 472.0621337890625
[step: 22] loss: 466.6416320800781
[step: 23] loss: 460.97039794921875
[step: 24] loss: 455.985595703125
[step: 25] loss: 452.32708740234375
[step: 26] loss: 450.2729797363281
[step: 27] loss: 449.74725341796875
[step: 28] loss: 450.39025878

ValueError: Cannot feed value of shape (0,) for Tensor 'Placeholder:0', which has shape '(?, 100, 2)'

In [5]:
# # plt.plot(dataY2, label='row data') # 데이터 있을때
time = xy2[:, 1];

# plt.scatter(time, test_predict, label='predict')
# plt.xlabel("Time Period")
# plt.ylabel("photometric")
# plt.xticks(np.arange(9, 19, step=1), ['9:00','10:00','11:00','12:00','13:00','14:00','15:00','16:00','17:00','18:00']) # v2 모델
# # plt.ylabel("cct")
# plt.grid(True)
# plt.legend(loc='upper right')
# plt.show()

In [6]:
bar_y = []

for i in range(len(test_predict)):
    bar_y.append(test_predict[i][0])

plt.bar(time, bar_y, tick_label=('9:00','10:00','11:00','12:00','13:00','14:00','15:00','16:00','17:00','18:00'), align='center', color='green');
plt.ylim(0, 1000)
plt.xlabel("Time Period")
plt.ylabel("photometric")
# plt.twiny()
plt.twinx()
plt.plot(time, bar_y, 'r.-')
plt.ylim(0, 1000)
plt.title("predicted")
plt.show()